<a href="https://colab.research.google.com/github/giuliocapecchi/IR_project/blob/main/IR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%pip install torch matplotlib nltk tqdm gdown ir_datasets humanize

# 1. Download and prepare the collection

In [2]:
# chosen_collection can be one of ["vaswani", "msmarco", "covid"]

chosen_collection = "vaswani"

In [3]:
import gdown
import ir_datasets
import pandas as pd
import os

if chosen_collection not in ["vaswani", "msmarco"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'msmarco']")

if chosen_collection == "msmarco":

    os.makedirs('./collection/msmarco', exist_ok=True)

    url_collection = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
    
    if not os.path.exists('./collection/msmarco/msmarco.tsv'):
        gdown.download(url_collection, './collection/msmarco/msmarco.tsv', quiet=False)
    
    """os.makedirs('./pickles', exist_ok=True)
    if not os.path.exists('./pickles/stats.pkl'):
        gdown.download(url_stats, './pickles/stats.pkl', quiet=False)
    if not os.path.exists('./pickles/lex.pkl'):
        gdown.download(url_lex, './pickles/lex.pkl', quiet=False)
    if not os.path.exists('./pickles/inv.pkl'):
        gdown.download(url_inv, './pickles/inv.pkl', quiet=False)
    if not os.path.exists('./pickles/doc.pkl'):
        gdown.download(url_doc, './pickles/doc.pkl', quiet=False)"""

elif chosen_collection == "vaswani":
    os.makedirs('./collection/vaswani', exist_ok=True)

    vaswani_dataset = ir_datasets.load(chosen_collection)
    docs = list(vaswani_dataset.docs_iter())
    df = pd.DataFrame(docs)
    df['doc_id'] = (df['doc_id'].astype(int) - 1).astype(str)
    # rimuovi i \n da ogni documento
    df['text'] = df['text'].str.replace('\n', ' ')
    if not os.path.exists('./collection/vaswani/vaswani.tsv'):
        df.to_csv('./collection/vaswani/vaswani.tsv', sep='\t', header=False, index=False)

Standard preprocessing but with the usage of the *PyStemmer* library.

In [4]:
import re
import string
import nltk
import Stemmer # PyStemmer


nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = Stemmer.Stemmer('english')
# stemmer = nltk.stem.PorterStemmer().stem # much slower than PyStemmer


def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand and special chars
    s = re.sub(r"[‘’´“”–-]", "'", s.replace("&", " and ")) # this replaces & with 'and' and normalises quotes
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # this removes dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = re.sub(r"\s+", " ", s).strip()
    # tokenisation
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # stemming
    return STEMMER.stemWords(tokens)

In [5]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [6]:
import pickle
import humanize
import os
from tqdm import tqdm

def print_pickled_size(var_name, var):
    # If the 'tmp' directory does not exist, we first create it
    os.makedirs('./tmp', exist_ok=True)
    with open(f"./tmp/{var_name}.pickle", 'wb') as f:
        pickle.dump(var, f)
    print(f'{var_name} requires {humanize.naturalsize(os.path.getsize(f"./tmp/{var_name}.pickle"))}')
    os.remove(f"./tmp/{var_name}.pickle")
    os.removedirs('./tmp')


def vbyte_encode(number):
    bytes_list = bytearray()
    while True:
        byte = number & 0x7F # Prendi i 7 bit meno significativi -> 0111 1111 = 0x7F
        number >>= 7 # Shifta a destra di 7 bit
        if number:
            bytes_list.append(byte) # Aggiungo i 7 bit al risultato
        else:
            bytes_list.append(0x80 | byte) # Aggiungo i 7 bit con il bit di continuazione, 0x80 = 1000 0000
            break
    return bytes(bytes_list)

def vbyte_decode(bytes_seq):
    number = 0
    for i, byte in enumerate(bytes_seq):
        number |= (byte & 0x7F) << (7 * i)
        if byte & 0x80:
            break
    return number

def decode_concatenated_vbyte(encoded_bytes):
    decoded_numbers = []
    current_number = 0
    shift_amount = 0
    
    for byte in encoded_bytes:
        if byte & 0x80:  # Bit di continuazione trovato, fine del numero
            current_number |= (byte & 0x7F) << shift_amount
            decoded_numbers.append(current_number)
            current_number = 0
            shift_amount = 0
        else:  # Continuo a comporre il numero
            current_number |= (byte & 0x7F) << shift_amount
            shift_amount += 7
    
    return decoded_numbers

#------------------------------------------------------------------------------------------------------------------------------------------------------------------#

def compress_index(lexicon, inv_d, inv_f):    
    compressed_inv_d = {}
    compressed_inv_f = {}

    for term, (termid, df, _) in tqdm(lexicon.items(), desc="Compressing lists", unit="term"):
        encoded_d = bytearray()
        for x in inv_d[termid]:
            encoded_d.extend(vbyte_encode(x)) 
        assert decode_concatenated_vbyte(encoded_d) == inv_d[termid]
        compressed_inv_d[termid] = encoded_d

        encoded_f = bytearray()
        for x in inv_f[termid]:
            encoded_f.extend(vbyte_encode(x))
        assert decode_concatenated_vbyte(encoded_f) == inv_f[termid]
        compressed_inv_f[termid] = encoded_f

    return compressed_inv_d, compressed_inv_f

## Functions to build the inverted index

In [7]:
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm

def build_index(filepath, batch_size=10000):
    total_documents = sum(1 for _ in open(filepath)) # get total number of documents

    lexicon = {}
    inv_d = {}
    inv_f = {}
    doc_index = []
    total_dl = 0
    num_docs = 0
    termid = 0

    with open(filepath, 'r') as file:        
        batch = []
        
        with tqdm(total=total_documents, desc="Processing documents", unit="doc") as pbar:
            for line in file:
                batch.append(line.strip())
                
                # when the batch is full, we process it
                if len(batch) >= batch_size:
                    for line in batch:
                        doc_id, text = line.split('\t', 1) # '1' specifies the number of splits
                        doc_id = int(doc_id)
                        tokens = preprocess(text)
                        token_tf = Counter(tokens)

                        for token, tf in token_tf.items():
                            if token not in lexicon:
                                lexicon[token] = [termid, 0, 0] # termid, df, tf
                                inv_d[termid], inv_f[termid] = [], [] # docids, freqs
                                termid += 1
                            token_id = lexicon[token][0]  # get termid
                            inv_d[token_id].append(doc_id)  # add doc_id to the list of documents containing the term
                            inv_f[token_id].append(tf)  # add term frequency for this doc
                            lexicon[token][1] += 1  # increment document frequency (df)
                            lexicon[token][2] += tf  # increment total term frequency (tf)

                        doclen = len(tokens)
                        doc_index.append((str(doc_id), doclen))
                        total_dl += doclen
                        num_docs += 1                    
                    # update progress bar for each processed document
                    pbar.update(len(batch))
                    batch = []

            # process the remaining documents in the last batch
            if batch:
                for line in batch:
                    doc_id, text = line.split('\t', 1)
                    doc_id = int(doc_id)
                    tokens = preprocess(text)
                    token_tf = Counter(tokens)

                    for token, tf in token_tf.items():
                        if token not in lexicon:
                            lexicon[token] = [termid, 0, 0]
                            inv_d[termid], inv_f[termid] = [], []
                            termid += 1
                        token_id = lexicon[token][0]  # get termid
                        inv_d[token_id].append(doc_id)  # get doc_id to the list of documents containing the term
                        inv_f[token_id].append(tf)  # get term frequency for this doc
                        lexicon[token][1] += 1  # increment document frequency (df)
                        lexicon[token][2] += tf  # increment total term frequency (tf)

                    doclen = len(tokens)
                    doc_index.append((str(doc_id), doclen))
                    total_dl += doclen
                    num_docs += 1                    
                    pbar.update(1)
                    
    stats = {
        'num_docs': num_docs,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [8]:
import math
import bisect


class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc
            self.total_docs_number = len(doc)


        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]


        ###################################################################################
        def score(self): # TODO : check if correct, this is TF-IDF score
            """
            Calculate TF-IDF score of the current document in the posting list.
            """
            if self.is_end_list():
                return math.inf 
            
            tf = self.freqs[self.pos]
                        
            if tf > 0:
                wtd = 1 + math.log(tf)
            else:
                wtd = 0 # avoid log(0)
            
            df = self.len()  # document frequency
            if df > 0:
                idf = math.log(self.total_docs_number / df)
            else:
                idf = 0  # avoid log(0)
            
            # finally calculate tf-idf score
            tfidf = wtd * idf

            return tfidf

        ###################################################################################

        def score_bm25(self): # TODO: check if correct. Also at the moment it is not used
            if self.is_end_list():
                return math.inf
            else:
                # Parametri standard di BM25
                b = 0.75
                k_1 = 1.5
                
                # Lunghezza del documento corrente
                dl = self.doc[self.docid()][1]
                
                # Lunghezza media del documento (deve essere calcolata globalmente)
                avdl = 25.09  # Assicurati di calcolare avdl correttamente
                
                # Calcola la frequenza del termine nel documento
                tf = self.freqs[self.pos]
                
                # Numero totale di documenti nella collezione
                N = self.total_docs_number
                
                # Numero di documenti che contengono il termine
                n = self.len()  # document frequency
                
                # Calcola l'IDF
                idf = math.log((N - n + 0.5) / (n + 0.5) + 1)
                
                # Calcola la normalizzazione del TF
                norm_tf = (tf * (k_1 + 1)) / (tf + k_1 * (1 - b + b * (dl / avdl)))
                
                # Calcola il punteggio BM25
                rsv_bm25 = idf * norm_tf
                
                return rsv_bm25
            
            ###################################################################################

        def next(self, target=None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    self.pos = bisect.bisect_left(self.docids, target, self.pos)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)
        

    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stats = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        return InvertedIndex.PostingListIterator( self.inv['docids'][termid], self.inv['freqs'][termid], self.doc)
    

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]

In [9]:
# import cProfile
# import pstats

# cProfile.run("build_index('./vaswani.tsv')", "output.prof")
# p = pstats.Stats("output.prof")
# p.sort_stats("cumtime").print_stats(10)
# os.remove("output.prof")

## Building the index on the chosen collection 

Now build up the index for the chosen collection. It is built only if a pickled version of its components doesn't exist already :

In [10]:
import pickle
import os

# If the 'pickles' directory does not exist, we first create it
os.makedirs('./pickles', exist_ok=True)

if chosen_collection == "msmarco":
    try: # try to open the pickled files, else build the index
        with open('./pickles/inv_index.pkl', 'rb') as f:
            inv_index = pickle.load(f)
        
    except FileNotFoundError:
        lex, inv, doc, stats = build_index('./collection/'+chosen_collection + '/'+chosen_collection+'.tsv')

        # Save the lexicon, inverted lists, and document index to disk
        with open('./pickles/lex.pkl', 'wb') as f:
            pickle.dump(lex, f)
        with open('./pickles/inv.pkl', 'wb') as f:
            pickle.dump(inv, f)
        with open('./pickles/doc.pkl', 'wb') as f:
            pickle.dump(doc, f)
        with open('./pickles/stats.pkl', 'wb') as f:
            pickle.dump(stats, f)
                    
        # Compress the inverted lists
        #inv['docids'], inv['freqs'] = compress_index(lex, inv['docids'], inv['freqs'])
        
        inv_index = InvertedIndex(lex, inv, doc, stats)
        with open('./pickles/inv_index.pkl', 'wb') as f:
            pickle.dump(inv_index, f)
else:
    lex, inv, doc, stats = build_index('./collection/'+chosen_collection + '/'+chosen_collection+'.tsv')
    inv_index = InvertedIndex(lex, inv, doc, stats)


print(f"Numero di documenti: {inv_index.num_docs()}")

Processing documents:   0%|          | 0/11429 [00:00<?, ?doc/s]

Numero di documenti: 11429


In [11]:
# # Create a test collection
# test_collection = [
#     "0\tterm1 term2 term3",
#     "1\tterm2 term3 term4",
#     "2\tterm1 term4 term5",
#     "3\tterm3 term5 term6",
#     "4\tterm1 term2 term3",
# ]

# # Write the fake collection to a temporary file
# with open('./collection/test_collection.tsv', 'w') as f:
#     for line in test_collection:
#         f.write(line + '\n')

# # Call the build_index function on the test collection
# lex, inv, doc, stats = build_index('./collection/test_collection.tsv')

# # Print the results
# print("Lexicon:", lex)
# print("Inverted Index (docids):", inv['docids'])
# print("Inverted Index (freqs):", inv['freqs'])
# print("Document Index:", doc)
# print("Statistics:", stats)

# inv_index = InvertedIndex(lex, inv, doc, stats)

# query = "term1 term2"
# termids = inv_index.get_termids(query.split())
# postings = inv_index.get_postings(termids)
# print(f"Query: '{query}', TermID: {termids}")
# for posting in postings:
#     while not posting.is_end_list():
#         print(f"docid: {posting.docid()}, score: {posting.score_bm25()}")
#         posting.next()


In [12]:
#print_pickled_size('inv_index', inv_index)

# 2. Download and prepare queries

In [13]:
import gzip

if chosen_collection not in ["vaswani", "msmarco"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'msmarco']")

if chosen_collection == "msmarco":
    if not os.path.exists('./collection/msmarco/msmarco-queries.tsv'):
        url = 'https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz'
        gdown.download(url, './collection/msmarco/msmarco-test2019-queries.tsv.gz', quiet=False)
        with gzip.open('./collection/msmarco/msmarco-test2019-queries.tsv.gz', 'rt') as f_in:
            with open('./collection/msmarco/msmarco-queries.tsv', 'w') as f_out:
                f_out.write(f_in.read())
        os.remove('./collection/msmarco/msmarco-test2019-queries.tsv.gz') # delete the compressed file
    queries = pd.read_csv('./collection/msmarco/msmarco-queries.tsv', sep='\t', header=None)
    queries.columns = ['qid', 'query']
    print("Number of queries: ",len(queries))

    if not os.path.exists('./collection/msmarco/msmarco-qrels.txt'):
        url = 'https://trec.nist.gov/data/deep/2019qrels-pass.txt'
        gdown.download(url, './collection/msmarco/msmarco-qrels.txt', quiet=False)
    qrels = pd.read_csv('./collection/msmarco/msmarco-qrels.txt', sep=' ', header=None)
    qrels.columns = ['qid', 'Q0', 'docid', 'rating']
    print("Number of relevance judgments: ",len(qrels))


elif chosen_collection == "vaswani":
    queries = pd.DataFrame(vaswani_dataset.queries_iter())
    queries.columns = ['qid', 'query']
    print("Number of queries: ",len(list(vaswani_dataset.queries_iter()))) 
    if not os.path.exists('./collection/vaswani/vaswani-queries.tsv'):
        queries.to_csv('./collection/vaswani/vaswani-queries.tsv', sep='\t', header=False, index=False)
    qrels = pd.DataFrame(vaswani_dataset.qrels_iter()) 
    qrels.columns = ['qid', 'docid', 'relevance', 'iteration']
    qrels['docid'] = (qrels['docid'].astype(int) - 1).astype(str) # convert to 0-based indexing

    if not os.path.exists('./collection/vaswani/vaswani-qrels.txt'):
        qrels.to_csv('./collection/vaswani/vaswani-qrels.txt', sep='\t', header=False, index=False)
    print("Number of relevance judgments: ",len(list(vaswani_dataset.qrels_iter())))

Number of queries:  93
Number of relevance judgments:  2083


In [14]:
from collections import namedtuple


class QueriesDataset:
    def __init__(self, df):
        self.queries = [Query(row.query_id, row.text) for row in df.itertuples()]

    def queries_iter(self):
        return iter(self.queries)

    def queries_count(self):
        return len(self.queries)
    
    def get_query(self, query_id):
        return self.queries[query_id]


Query = namedtuple('Query', ['query_id', 'text'])
queries.columns = ['query_id', 'text']
queries_dataset = QueriesDataset(queries)
print("The number of queries is: ", queries_dataset.queries_count())

The number of queries is:  93


Let's prepare the functions necessary to perform TAAT and DAAT query processing

First, we need a TopQueue class, which stores the top  K  (score, docid) tuples, using an heap 

In [15]:
import heapq

class TopQueue:
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

#print(sorted(topq.queue, reverse=True)) # print the queue sorted by score

### TAAT

In [16]:
from collections import defaultdict

def taat(postings, k=10):
    A = defaultdict(float)
    for posting in postings:
        current_docid = posting.docid()
        while current_docid != math.inf:
            A[current_docid] += posting.score()
            posting.next()
            current_docid = posting.docid()
    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)
    return sorted(top.queue, reverse=True)


def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return taat(postings)

### DAAT

In [17]:
import math

def min_docid(postings):
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat(postings, k=10):
    top = TopQueue(k)
    current_docid = min_docid(postings)
    while current_docid != math.inf:
        score = 0
        next_docid = math.inf
        for posting in postings:
            if posting.docid() == current_docid:
                score += posting.score()
                posting.next()
            if not posting.is_end_list():
                next_docid = posting.docid()
        top.insert(current_docid, score)
        current_docid = next_docid
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return daat(postings)

In [18]:
from tqdm import tqdm

@profile
def query_processing(queries_iter, fn):
    #for q in tqdm(queries_iter, desc="Processing queries", total=queries_dataset.queries_count(), unit="query"):
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res = fn(postings)

In [19]:
# import cProfile
# import pstats

# cProfile.run("query_processing(queries_dataset.queries_iter(), taat)", "./perfm/result.prof")
# p = pstats.Stats("./perfm/result.prof")
# p.sort_stats("cumtime").print_stats(25)

In [20]:
query_processing(queries_dataset.queries_iter(), taat)

query_processing (874.684 ms)


In [21]:
query_processing(queries_dataset.queries_iter(), daat)

query_processing (3058.723 ms)


---

# 3. Evaluation

A relevance assessment (called ***qrel*** in `ir_datasets`) is composed by:
* a **topic id** (called *query_id* in `ir_datasets`) as in a topic,
* a **docno** (called *doc_id* in `ir_datasets`) as in a document,
* a **judgement** (called *relevance* in `ir_datasets`) as a binary or graded relevance judgment/label, and
* an **iteration**, **UNUSED** and always equal to the string `'0'`.

In [22]:
# get the qrels for the chosen collection
qrels = pd.read_csv('./collection/'+chosen_collection+'/'+chosen_collection+'-qrels.txt', sep='\t', header=None)
qrels.columns = ['query_id', 'doc_id', 'relevance', 'iteration']
print("Number of relevance judgments: ",len(qrels))

Number of relevance judgments:  2083


In [23]:
import os

def create_run_file(queries_iter, fn, run_id, output_file):
    """
    Preprocess the queries and write the results to a run file.
    :param queries_iter: Query iterator
    :param fn: Function to process the postings and return the results in the format (score, docid)
    :param run_id: Name identifier for the run
    :param output_file: Output run file
    """
    if not os.path.exists('./results'):
        os.makedirs('./results')
    with open(f"./results/{output_file}", "w") as f:
        for q in queries_iter:
            topic_id = q.query_id 
            query = preprocess(q.text)
            termids = inv_index.get_termids(query)
            postings = inv_index.get_postings(termids)
            results = fn(postings, k=100)
            
            if results:
                # Write results to the run file
                for rank, (score, docno) in enumerate(results, start=1):
                    line = f"{topic_id}\tQ0\t{docno}\t{rank}\t{score:.6f}\t{run_id}\n"
                    f.write(line)
            else:
                # Annotate that no results were found for this query
                line = f"{topic_id}\tQ0\tNO_RESULTS\t0\t0.0\t{run_id}\n"
                f.write(line)

    print(f"Run file {output_file} produced successfully.")

In [24]:
create_run_file(queries_dataset.queries_iter(), taat, "run", f"{chosen_collection}.run")

Run file vaswani.run produced successfully.


In [25]:
# open the qrel file and create a 'gold' dictionary that contains for each qid the list of relevant documents
gold = {}
with open('./collection/vaswani/vaswani-qrels.txt', 'r') as f:
    for line in f:
        qid, docid, rel, _ = line.strip().split('\t')
        qid = int(qid)
        docid = int(docid)
        if int(rel) > 0:
            if qid not in gold:
                gold[qid] = []
            gold[qid].append(docid)


D = list(range(inv_index.num_docs())) # from 0 to number of documents
T = list(range(1,len(gold))) # from 1 to number of queries

# open the run file and create a 'results' dictionary that contains for each qid the list of retrieved documents
results = {}
with open('./results/vaswani.run', 'r') as f:
    for line in f:
        qid, _, docid, rank, score, _ = line.strip().split('\t')
        qid = int(qid)
        if docid == 'NO_RESULTS':
            docid = -1
        else:
            docid = int(docid)
        if qid not in results:
            results[qid] = []
        results[qid].append(docid)


A = {} # A will contain if the results returned by the IR system are actually relevant
for qid in gold:
    A[qid] = [1 if d in gold[qid] else 0 for d in D]



print("Number of queries in the qrel file: ", len(gold))
print("Number of queries in the run file: ", len(results))
print("Number of 'A': ", len(A))

# print("qrels:\t",gold)
# print("results:",results)
# # stampa i primi 10 elementi di A
# print("A:\t", {k: A[k] for k in list(A)[:10]})

Number of queries in the qrel file:  93
Number of queries in the run file:  93
Number of 'A':  93


In [26]:
def precision(results, qrels): # i qrels sono i relevance assesment files
    retrieved = len(results)
    retrieved_relevant = sum([qrels[d] for d in results])
    return retrieved_relevant/retrieved # effettivamente torna

def recall(results, qrels):
    relevant = sum(qrels)
    retrieved_relevant = sum([qrels[d] for d in results])
    return retrieved_relevant/relevant

def f1_score(results, qrels):
    P = precision(results, qrels)
    R = recall(results, qrels)
    # TODO : rimuovi perchè non esiste che sia zero
    if P + R == 0:
        return 0
    ##########################################
    return 2*R*P/(R+P)
    
for t in T:
    print(f'Topic {t}: F1 = {f1_score(results[t], A[t])}')

Topic 1: F1 = 0.18487394957983191
Topic 2: F1 = 0.1391304347826087
Topic 3: F1 = 0.25563909774436094
Topic 4: F1 = 0.07619047619047618
Topic 5: F1 = 0
Topic 6: F1 = 0.1090909090909091
Topic 7: F1 = 0.5371428571428571
Topic 8: F1 = 0.019801980198019802
Topic 9: F1 = 0.0392156862745098
Topic 10: F1 = 0.10810810810810809
Topic 11: F1 = 0.07692307692307693
Topic 12: F1 = 0.3597122302158273
Topic 13: F1 = 0.31446540880503143
Topic 14: F1 = 0.3974358974358974
Topic 15: F1 = 0.2878787878787879
Topic 16: F1 = 0.09523809523809523
Topic 17: F1 = 0.2601626016260163
Topic 18: F1 = 0.14159292035398233
Topic 19: F1 = 0.24193548387096772
Topic 20: F1 = 0.22764227642276424
Topic 21: F1 = 0.4133333333333334
Topic 22: F1 = 0.3393939393939394
Topic 23: F1 = 0.19999999999999998
Topic 24: F1 = 0.2877697841726619
Topic 25: F1 = 0.3236994219653179
Topic 26: F1 = 0.32679738562091504
Topic 27: F1 = 0.25
Topic 28: F1 = 0.1111111111111111
Topic 29: F1 = 0.07207207207207207
Topic 30: F1 = 0.09345794392523366
Topi

In [27]:
import numpy as np

def precision_at_k(results, qrels, k):
    return precision(results[:k], qrels) # prende i primi k risultati e calcola la precisione su questi

for k in [1, 3, 5, 10, 20]:
    print(f'P@{k} = {np.mean([precision_at_k(results[t], A[t], k) for t in T])}') # media delle precisioni per ogni topic


def recall_at_k(results, qrels, k):
    return recall(results[:k], qrels)

for k in [1, 3, 5, 10, 20]:
    print(f'R@{k} = {np.mean([recall_at_k(results[t], A[t], k) for t in T])}') # stessa cosa di prima ma per recall

P@1 = 0.5652173913043478
P@3 = 0.4239130434782608
P@5 = 0.3717391304347826
P@10 = 0.30869565217391304
P@20 = 0.2510869565217391
R@1 = 0.037321107090456586
R@3 = 0.08939794437335334
R@5 = 0.1286306945945357
R@10 = 0.19287334802568679
R@20 = 0.3014126920954252


In [28]:
import ir_measures
from ir_measures import * # import natural measure names

chosen_collection = "vaswani"
qrels = pd.read_csv(f'./collection/{chosen_collection}/{chosen_collection}-qrels.txt', sep='\t', header=None, names=['query_id', 'doc_id', 'relevance', 'iteration'])
qrels['query_id'] = qrels['query_id'].apply(str)
qrels['doc_id'] = qrels['doc_id'].apply(str)
qrels['relevance'] = qrels['relevance'].apply(int)


run_file = list(ir_measures.read_trec_run(f'./results/{chosen_collection}.run'))


print(ir_measures.calc_aggregate([P@5, P(rel=2)@5, nDCG@10, AP, AP(rel=2), Bpref, Bpref(rel=2), Judged@10], qrels, run_file))

{P(rel=2)@5: 0.0, P@5: 0.37204301075268803, AP(rel=2): 0.0, Judged@10: 0.30752688172043, Bpref(rel=2): 0.0, Bpref: 0.5969221205233392, AP: 0.21990222074835353, nDCG@10: 0.3805876017994083}


In [29]:
# TODO : da rivedere

count = 0
for metric in ir_measures.iter_calc([P@5, P(rel=2)@5, nDCG@10, AP, AP(rel=2), Bpref, Bpref(rel=2), Judged@10], qrels, run_file):
  print(metric)
  count += 1
  if count >= 10: break # only show top 10 items

run_file_p_rel2_5 = {m.query_id: m.value for m in ir_measures.iter_calc([Bpref(rel=2)], qrels, run_file)}

from scipy.stats import ttest_rel
qids = list(run_file_p_rel2_5.keys())
#ttest_rel([run_file_p_rel2_5[v] for v in qids])


Metric(query_id='1', measure=AP(rel=2), value=0.0)
Metric(query_id='1', measure=Bpref(rel=2), value=0.0)
Metric(query_id='1', measure=P(rel=2)@5, value=0.0)
Metric(query_id='1', measure=nDCG@10, value=0.4666234168001789)
Metric(query_id='2', measure=AP(rel=2), value=0.0)
Metric(query_id='2', measure=Bpref(rel=2), value=0.0)
Metric(query_id='2', measure=P(rel=2)@5, value=0.0)
Metric(query_id='2', measure=nDCG@10, value=0.06943122193677727)
Metric(query_id='3', measure=AP(rel=2), value=0.0)
Metric(query_id='3', measure=Bpref(rel=2), value=0.0)


---

# Trying it out

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time


# UI elements
search_bar = widgets.Text(
    placeholder='Type in a query...',
    description='Search:',
    layout=widgets.Layout(width='80%')
)
search_button = widgets.Button(
    description='Search',
    button_style='success',
    tooltip='Execute the query',
    icon='search'
)

score_function_rbtn = widgets.RadioButtons(options=['TF-IDF', 'BM25'], description='Scoring function:', disabled=False)
algo_rbtn = widgets.RadioButtons(options=['TAAT', 'DAAT'], description='Algorithm:', disabled=False)
_style = widgets.HTML(
    "<style>.widget-radio-box {flex-direction: row !important;}.widget-radio-box"
    " label{margin:2px !important;width: 100px !important;}</style>",
    layout=widgets.Layout(display="none"),
)

output_area = widgets.Output()


def on_search_click(b):
    with output_area:
        clear_output()  # clean previous output
        query = search_bar.value
        if not query.strip():
            print("Please, type in a query.")
            return
        
        selected_scoring_function = score_function_rbtn.value
        print(f"Selected scoring function: {selected_scoring_function}")
        # TODO : implement the selected scoring function

        selected_algorithm = algo_rbtn.value
        print(f"Selected Algorithm: {selected_algorithm}")
        
        start_time = time.time()
        # --- QUERY EXECUTION ---
        processed_query = preprocess(query)
        termids = inv_index.get_termids(processed_query)
        postings = inv_index.get_postings(termids)
        
        if selected_algorithm == 'TAAT':
            results = taat(postings)
        else:
            results = daat(postings)
        # ------------------------
        elapsed_time = (time.time() - start_time) * 1000 # convert in ms

        # finally show the results
        print(f"Found: {len(results)} documents\n")
        for res in results:
            res = (round(res[0], 4), res[1]) # TODO : si potrebbe spostare direttamente nella score function
            print(f" - {res}")
        print(f"\nExecution time: {elapsed_time:.2f} ms")

search_button.on_click(on_search_click)
# link search button to the enter key
search_bar.continuous_update = False
search_bar.observe(on_search_click, names='value')

top_row = widgets.HBox([score_function_rbtn,_style])
middle_row = widgets.HBox([algo_rbtn,_style])
bottom_row = widgets.HBox([search_bar, search_button])

# finally display the UI
display(widgets.VBox([top_row,middle_row, bottom_row]))
display(output_area)


Output()